# Pruebas Trabajo con MLPs

In [10]:
from keras.optimizers import SGD
from tensorflow import keras
import cv2
import numpy as np
import os
from sklearn.cluster import MiniBatchKMeans
from keras import Sequential
from sklearn.base import BaseEstimator, TransformerMixin
from keras.layers import Dense, Input, Dropout, Flatten
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [2]:
dataset_path = '/home/pablo/Desktop/tercero/mdp/trabajo/HANDS'   # Cambiar por la carpeta en la que se encuentran las imágenes
 
jpgFiles = [fJPG for fJPG in os.listdir(dataset_path) if fJPG.endswith('.jpg')]

bufferImages = []
y = []

# Extraer 50 patches aleatorios de cada imagen
for filename in jpgFiles:
    img_path = os.path.join(dataset_path, filename)
    label = filename.split('.')[0][-2:]
    y.append(label)
    with open(img_path) as infile:
        # Obtener etiqueta
        img_label = filename[5:7] 
        # Leer imagen
        img = cv2.imread(img_path)
        # Convertir imagen de BGR a RGB
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        bufferImages.append(img_gray)
print(len(bufferImages))
print('Comprobación:', all([x in ['RB', 'RF', 'LB', 'LF'] for x in y]))

200
Comprobación: True


In [3]:
class CustomPreprocessorSamePatches(BaseEstimator, TransformerMixin):
    def __init__(self, patchSize=10, nClusters = 81, max_patches=50):
        self.patchSize = patchSize
        self.nClusters = nClusters
        self.max_patches = max_patches

    def fit(self, X, y=None):
        return self       
    
    def extract_centric_patches(self, img, patch_size, num_patches, seed=42):
        assert img.shape == (150, 150), "La imagen debe tener resolución 150x150."
        assert patch_size > 0, "El tamaño del patch debe ser mayor que 0."
        assert patch_size * num_patches <= 150 * 150, "Demasiados patches para esta resolución."
        
        # Calcular la cuadrícula de celdas no superpuestas
        grid_rows = 150 // patch_size
        grid_cols = 150 // patch_size
        total_cells = grid_rows * grid_cols

        assert num_patches <= total_cells, "Demasiados patches para el tamaño del patch y la resolución de la imagen."
        
        # Generar las posiciones de la cuadrícula
        all_positions = [(i * patch_size, j * patch_size) for i in range(grid_rows) for j in range(grid_cols)]

        # Calcular las distancias de las celdas al centro de la imagen
        center = (150 // 2, 150 // 2)
        distances = [
            np.sqrt((row + patch_size // 2 - center[0])**2 + (col + patch_size // 2 - center[1])**2)
            for row, col in all_positions
        ]

        # Ordenar las posiciones por cercanía al centro
        sorted_positions = [pos for _, pos in sorted(zip(distances, all_positions))]
        
        # Seleccionar los patches más céntricos
        np.random.seed(seed)
        selected_positions = sorted_positions[:num_patches]
        
        # Extraer los patches en las posiciones seleccionadas
        patches = [img[row:row + patch_size, col:col + patch_size] for row, col in selected_positions]

        #self.visualize_disjoint_patches(img, patches, selected_positions, patch_size)
        
        return np.array(patches)


    def bovw(self, images, patchSize, nClusters, maxPatches):
        bufferData = []
        for i in images:
            resized_img = cv2.resize(i, (150, 150), interpolation=cv2.INTER_AREA)
            patches = self.extract_centric_patches(resized_img, patchSize, num_patches=maxPatches, seed=27)
            patches = np.reshape(patches, (len(patches), -1))
            bufferData.append(patches)
            
        bufferData = np.array(bufferData)
        print(bufferData.shape)
        return bufferData

    def transform(self, images):
        # Aplica tu función de preprocesamiento aquí
        # Por ejemplo:
        # return your_custom_function(X, param1=self.param1)
        return self.bovw(images, self.patchSize, self.nClusters, self.max_patches)  # Devuelve los datos preprocesados

In [ ]:
cp = CustomPreprocessorSamePatches()
resul = cp.transform(bufferImages)

(200, 50, 100)


array([[[135, 135, 130, ..., 131, 128, 120],
        [128, 125, 116, ..., 125, 124, 122],
        [114, 113, 118, ..., 105,  70,  54],
        ...,
        [129, 126, 123, ..., 126, 122, 119],
        [178, 172, 180, ..., 187, 181, 164],
        [140, 138, 137, ...,  97,  99, 100]],

       [[134, 133, 132, ..., 128, 126, 125],
        [134, 132, 134, ..., 130, 130, 130],
        [114, 114, 119, ..., 127, 128, 129],
        ...,
        [162, 166, 169, ..., 170, 169, 168],
        [ 90,  94,  98, ...,  92,  96, 104],
        [161, 162, 162, ..., 127, 126, 125]],

       [[123, 124, 126, ..., 132, 130, 128],
        [121, 117, 121, ..., 124, 123, 122],
        [127, 128, 129, ..., 122, 124, 127],
        ...,
        [140, 140, 144, ..., 137, 137, 137],
        [149, 152, 153, ..., 147, 147, 147],
        [ 58,  74,  88, ..., 111, 128, 131]],

       ...,

       [[162, 162, 167, ..., 201, 200, 201],
        [199, 197, 185, ..., 189, 190, 190],
        [160, 161, 161, ..., 196, 196, 196

In [7]:
model = Sequential()
model.add(Input(shape=(50, 100)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer = SGD(), loss='categorical_crossentropy',
                metrics=['accuracy'])

In [24]:
y_one_hot = to_categorical(y, 4)
x_train, x_test, y_train, y_test = train_test_split(resul, y_one_hot, test_size=0.2)

history = model.fit(x_train, y_train, epochs=60, batch_size=16)

ValueError: invalid literal for int() with base 10: 'RB'

In [18]:
y_train.shape

(40, 50, 100)